In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import pickle
import json

In [43]:
import requests
from bs4 import BeautifulSoup
import json
import pickle

# URL of the website
base_url = "https://bulbapedia.bulbagarden.net/wiki/"

# Load the Pokémon list from file
with open('pokemon_list.pkl', 'rb') as f:
    pokemon_list = pickle.load(f)

# Limit the number of Pokémon to scrape
stop_index = -1  # Change this as needed

# Dictionary to store evolution data
evolution_dict = {}
# Define how often to save (every N Pokémon)
save_interval = 5
scraped_count = 0

# Loop through the Pokémon list and scrape data
for pokemon in pokemon_list[:stop_index]:
    print(pokemon)
    url = base_url + pokemon
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to retrieve page for {pokemon}. Skipping.")
        continue

    wait_time = random.randint(1, 3)
    print(f"Waiting for {wait_time} seconds...")
    time.sleep(wait_time)

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Locate the div containing evolution data
    #div = soup.find("div", style="float:center; display:flex; flex-flow:row wrap;")
    div = soup.find("div", style=lambda value: value and ("float:center; display:flex; flex-flow:row wrap;" in value or "display: flex; flex-wrap: wrap; justify-content: space-evenly" in value))
    if div:
        # Find the parent table
        parent_table = div.find("table")
        if parent_table:
            evolution_data = []  # List to store data for the current Pokémon
            # Find all nested tables within the parent table
            #nested_tables = parent_table.find_all(
            #    "table",
            #    style="margin:auto; text-align:center; background: #3FA129; border-radius: 10px; -moz-border-radius: 10px; -webkit-border-radius: 10px; -khtml-border-radius: 10px; -icab-border-radius: 10px; -o-border-radius: 10px;"
            #)
            nested_tables = parent_table.find_all("table")
            for nested_table in nested_tables:
                # Extract evolution stage from the <small> tag
                evolution_stage = nested_table.find("small").get_text(strip=True) if nested_table.find("small") else None

                # Extract Pokémon name
                name_cell = nested_table.find("a", href=True)
                if name_cell and "href" in name_cell.attrs:
                    href_value = name_cell["href"]
                    if "File:" in href_value:
                        pokemon_name = href_value.split("File:")[1].split(".png")[0]
                        # Clean the Pokémon name (remove leading digits and convert to lowercase)
                        pokemon_name = pokemon_name.lstrip("0123456789").lower()
                        evolution_data.append({
                            "evolution_stage": evolution_stage,
                            "pokemon_name": pokemon_name
                        })

            # Add data to the dictionary under the base form's name
            if evolution_data:
                #base_form = evolution_data[0]["pokemon_name"]  # Assume the first entry is the base form
                evolution_dict[pokemon] = evolution_data
        else:
            print(f"Parent table not found for {pokemon}.")
    else:
        print(f"No div found for {pokemon}, falling back to table scraping.")
        
        # Fallback: If no div is found, scrape data directly from the table
        # Locate the table directly in the page
        tables = soup.find_all("table", style="margin:auto; text-align:center; background: #3FA129; border-radius: 10px; -moz-border-radius: 10px; -webkit-border-radius: 10px; -khtml-border-radius: 10px; -icab-border-radius: 10px; -o-border-radius: 10px;")
        for table in tables:
            # Iterate through the rows in the table and extract evolution data
            rows = table.find_all("tr")
            for row in rows:
                cells = row.find_all("td")
                if len(cells) > 1:
                    # Extract evolution stage from the <small> tag (if present)
                    evolution_stage = cells[0].get_text(strip=True) if cells[0].find("small") else None
                    name_cell = cells[1]
                    if name_cell:
                        link_tag = name_cell.find("a", href=True)
                        if link_tag and "href" in link_tag.attrs:
                            href_value = link_tag["href"]
                            # Extract the portion after "File:" and before ".png"
                            if "File:" in href_value:
                                pokemon_name = href_value.split("File:")[1].split(".png")[0]  # e.g., "0003Venusaur"
                                # Remove leading digits and convert to lowercase
                                pokemon_name = pokemon_name.lstrip("0123456789").lower()
                                evolution_data.append({
                                    "evolution_stage": evolution_stage,
                                    "pokemon_name": pokemon_name
                                })
        if evolution_data:
                #base_form = evolution_data[0]["pokemon_name"]  # Assume the first entry is the base form
                evolution_dict[pokemon] = evolution_data
    # Store evolution data
    evolution_dict[pokemon] = evolution_data
    scraped_count += 1

    # Periodically save the data to a JSON file
    if scraped_count % save_interval == 0:
        with open('evolution_data.json', 'w') as f:
            json.dump(evolution_dict, f, indent=4)
        print(f"Saved data after scraping {scraped_count} Pokémon.")

# Save the scraped evolution data to a JSON file
with open('evolution_data.json', 'w') as f:
    json.dump(evolution_dict, f, indent=4)

print("Evolution data saved successfully.")


bulbasaur
Waiting for 2 seconds...
ivysaur
Waiting for 3 seconds...
venusaur
Waiting for 1 seconds...
charmander
Waiting for 2 seconds...
charmeleon
Waiting for 1 seconds...
Saved data after scraping 5 Pokémon.
charizard
Waiting for 2 seconds...
squirtle
Waiting for 1 seconds...
wartortle
Waiting for 2 seconds...
blastoise
Waiting for 3 seconds...
caterpie
Waiting for 3 seconds...
Saved data after scraping 10 Pokémon.
metapod
Waiting for 1 seconds...
butterfree
Waiting for 2 seconds...
weedle
Waiting for 3 seconds...
kakuna
Waiting for 1 seconds...
beedrill
Waiting for 2 seconds...
Saved data after scraping 15 Pokémon.
pidgey
Waiting for 2 seconds...
pidgeotto
Waiting for 2 seconds...
pidgeot
Waiting for 3 seconds...
rattata
Waiting for 3 seconds...
raticate
Waiting for 3 seconds...
Saved data after scraping 20 Pokémon.
spearow
Waiting for 2 seconds...
fearow
Waiting for 1 seconds...
ekans
Waiting for 1 seconds...
arbok
Waiting for 1 seconds...
pikachu
Waiting for 1 seconds...
Saved da